Importing data from Google Sheets

Referencing snippet from here: https://colab.research.google.com/notebooks/snippets/sheets.ipynb#scrollTo=JiJVCmu3dhFa 

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
worksheet = gc.open('allActivisionBlizzardGlassdoor').sheet1

In [3]:
rows = worksheet.get_all_values()
print(rows)

[['', '', '', '', '', ''], ['title', 'author_info', 'rating', 'pros', 'cons', 'helpful'], ['Overall, great place', 'Sep 21, 2021 - Senior Web Developer in Boston, MA', '5', 'good people great benefits everyone games any type of game you play', "hard work rushed deadlines bosses don't work as much as developers", 'Be the first to find this review helpful'], ['Fun, but overworked.', 'May 18, 2022 - Functional Tester', '3', 'Met some pretty incredible people. Can actually see your effects on the game.', "Too many hours. Disorganized. Corprate doesn't care about those under them.", 'Be the first to find this review helpful'], ['Terrible Managers', 'Apr 11, 2022 - Anonymous Employee', '1', 'Will say the position is wfh but then when you are hired will change their minds.', 'They send you free stuff sometimes.', '1 person found this review helpful'], ['Great People - Weak Pay', 'Mar 3, 2022 - Specialist in Santa Monica, CA', '3', 'Awesome time to be in gaming. Working on big titles is fun an

Converting the spreadsheet to a Pandas Dataframe

In [4]:
import pandas as pd

In [6]:
activision_df = pd.DataFrame.from_records(rows)
display(activision_df)

,0,1,2,3,4,5
0,,,,,,
1,title,author_info,rating,pros,cons,helpful
2,"Overall, great place","Sep 21, 2021 - Senior Web Developer in Boston, MA",5,good people great benefits everyone games any ...,hard work rushed deadlines bosses don't work a...,Be the first to find this review helpful
3,"Fun, but overworked.","May 18, 2022 - Functional Tester",3,Met some pretty incredible people. Can actuall...,Too many hours. Disorganized. Corprate doesn't...,Be the first to find this review helpful
4,Terrible Managers,"Apr 11, 2022 - Anonymous Employee",1,Will say the position is wfh but then when you...,They send you free stuff sometimes.,1 person found this review helpful
...,...,...,...,...,...,...
449,Amazing company,"Jan 22, 2022 - Anonymous Employee",5,"Great pay, Great hours, Great people","High speed, Time consuming, Long hours",Be the first to find this review helpful
450,Great Colleagues,"Jan 18, 2022 - Art Manager in Stockholm, Stock...",4,Wonderful People Work Life Balance Experienced...,Negativity because of scandals Poor communicat...,Be the first to find this review helpful
451,Tester,"Jan 25, 2022 - Anonymous Employee",3,Friendly colleges Interesting job tasks,Shifts no work life balance,Be the first to find this review helpful
452,Localization QA Tester,"Jan 24, 2022 - Localization QA Tester in Dubli...",2,Good work place and nice people,low salary and no long contract,Be the first to find this review helpful


In [9]:
# Designating the first row of the dataframe as the header
activision_df.columns = activision_df.iloc[0]
activision_df = activision_df[1:]
activision_df.head()

1,title,author_info,rating,pros,cons,helpful
2,"Overall, great place","Sep 21, 2021 - Senior Web Developer in Boston, MA",5,good people great benefits everyone games any ...,hard work rushed deadlines bosses don't work a...,Be the first to find this review helpful
3,"Fun, but overworked.","May 18, 2022 - Functional Tester",3,Met some pretty incredible people. Can actuall...,Too many hours. Disorganized. Corprate doesn't...,Be the first to find this review helpful
4,Terrible Managers,"Apr 11, 2022 - Anonymous Employee",1,Will say the position is wfh but then when you...,They send you free stuff sometimes.,1 person found this review helpful
5,Great People - Weak Pay,"Mar 3, 2022 - Specialist in Santa Monica, CA",3,Awesome time to be in gaming. Working on big t...,Bad news headlines and lack of clear upward mo...,1 person found this review helpful
6,Poor Culture,"Feb 3, 2022 - Marketing Manager in Santa Monic...",2,Gaming benefits if that's your thing,Lack of any company culture,Be the first to find this review helpful


Cleaning up the dataframe by...

- Removing the `helpful` (last) column, which indicates how many Glassdoor users rated a review as "helpful." This information is not relevant to us.
- Parsing the date from the `author_info` (second) column. While job titles are not always provided – as review authors are not obligated by Glassdoor to state them — they are spliced out whenever they are. This allows us to only access the information we need: the date the review was posted.

In [10]:
# Remove last column by its title
activision_df = activision_df.drop('helpful', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [11]:
import datetime

In [12]:
# Helper function for date formatting (MM/DD/YY).
# i.e. Takes "Jan 1, 2000" as input, and returns "01/01/2000" as output.
# Note that both I/O are strings.
def format_date(original_date):

  date_components = original_date.split(' ')

  # Convert the month from abbreviated to numerical format.
  # Pad zeroes wherever appropriate.
  month_published = str(datetime.datetime.strptime(date_components[0], "%b").month).zfill(2)

  # Remove the trailing comma from the day (second item in list).
  # Again, pad zeroes wherever appropriate.
  day_published = date_components[1][0:-1].zfill(2)

  year_published = date_components[2]

  date_formatted = month_published + '/' + day_published + '/' + year_published
  return date_formatted

In [14]:
# Extract the date from the last column
for index, row in activision_df.iterrows():

  delimiter = ' - '
  split_info = row['author_info'].split(delimiter)
  
  date_published = ''

  # If a job title was provided by the reviewer, we splice it out.
  if len(split_info) > 1:
    date_published = split_info[0]

  # If no job title was provided, then the date is simply
  # the first item in the list, with the trailing space & hyphen excluded.
  # So exclude the last two characters.
  else:
    date_published = split_info[0][0:-2]
  
  # Format the date, relying on the helper function above.
  date_formatted = format_date(date_published)
  
  # Update the dataframe.
  activision_df.loc[index, 'author_info'] = date_formatted

In [15]:
activision_df.head()

1,title,author_info,rating,pros,cons
2,"Overall, great place",09/21/2021,5,good people great benefits everyone games any ...,hard work rushed deadlines bosses don't work a...
3,"Fun, but overworked.",05/18/2022,3,Met some pretty incredible people. Can actuall...,Too many hours. Disorganized. Corprate doesn't...
4,Terrible Managers,04/11/2022,1,Will say the position is wfh but then when you...,They send you free stuff sometimes.
5,Great People - Weak Pay,03/03/2022,3,Awesome time to be in gaming. Working on big t...,Bad news headlines and lack of clear upward mo...
6,Poor Culture,02/03/2022,2,Gaming benefits if that's your thing,Lack of any company culture


# Zero-shot classification

Now, we're going to use zero-shot classification to classify our reviews
according to these axes: Culture and Values, Diversity and Inclusion, Work/Life Balance, Senior Management, Compensation and Benefits, and Career Opportunities. 

To do this, we'll first make a long list of all the sentences from our reviews. 

Then, we'll use BART from Hugging Face (https://huggingface.co/facebook/bart-large-mnli) to classify those sentences, putting them into appropriate lists! We will also keep them separated by negative and positive by assuming that whatever is under "pros" is positive, and whatever is under "cons" can be expected to be negative--this will be useful later on when we begin to use BERT for sentiment analysis.

In [16]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 9.9 MB/s 
     |████████████████████████████████| 6.6 MB 52.5 MB/s 
     |████████████████████████████████| 596 kB 48.8 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [17]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [18]:
reviewsPro = activision_df['pros']      # pros column
reviewsCon = activision_df['cons']      # cons column

Now that we have our data, we need to parse things into sentences. This is a bit tricky--there's no unified format for user reviews. Some people use periods to separate sentences. Some use hyphens. Some use both hyphens and periods, with additional hyphens. When the data is scraped from the Internet, newlines are obliterated, so we can't use those as clues.

Instead, we'll assume that if we encounter a period or an exclamation mark, we're ending a sentence. We'll also assume that if we encounter a hyphen with a space after it, we're ending a sentence. 

In [19]:
proSentences = []
conSentences = []

def parseSentencesFromReviewsColumn(reviewsColumn, reviewsList):
  # Iterate through the review at each row in the given column.
  for (columnName, columnData) in reviewsColumn.iteritems():
    currSent = ""
    prevChar = columnData[0]
    # columnData = one full positive review section. Loop thru char by char
    for character in columnData:
      # If we encounter a period, assume sentence, but only if the built string is
      # not spaces and not empty.
      if character == '.':
        if currSent.strip("-. ") and not currSent.strip(",. ").isspace():
          reviewsList.append(currSent.strip("-."))
        currSent = ""
      # If we encounter a "- ", assume sentence.
      elif character == ' ' and prevChar == '-':
        if currSent.strip("-. ") and not currSent.strip(",. ").isspace():
          reviewsList.append(currSent.strip(".-"))
      
        currSent = ""
    
      # Keep building the string and keep track of the prev char.
      currSent += character
      prevChar = character
      
    # Append whatever was left if it wasn't already caught
    if currSent.strip("-. ") and not currSent.strip(",. ").isspace():
      reviewsList.append(currSent.strip(".-"))

# Run on positive and negative reviews
parseSentencesFromReviewsColumn(reviewsPro, proSentences)
parseSentencesFromReviewsColumn(reviewsCon, conSentences)

# Sanity check
print(proSentences[:15])
print(conSentences[:15])

['good people great benefits everyone games any type of game you play', 'Met some pretty incredible people', ' Can actually see your effects on the game', 'Will say the position is wfh but then when you are hired will change their minds', 'Awesome time to be in gaming', ' Working on big titles is fun and carries some prestige', ' Really great people to work with and a lot of fun', "Gaming benefits if that's your thing", 'Great IP, passionate people, Bagel Mondays and Donut Fridays', 'good coworkers, and nice group events', 'You get free games snack and all the coffee you can drink', ' Sometimes they order pizza for the office', ' Woohoo -_', 'Fun atmosphere at times depending on the people during your shift', '+ Work from home opportunities']
["hard work rushed deadlines bosses don't work as much as developers", 'Too many hours', ' Disorganized', " Corprate doesn't care about those under them", 'They send you free stuff sometimes', 'Bad news headlines and lack of clear upward mobility 

Now we're all set up to classify our sentences. We'll sort them into lists according to their valence and category (the six categories are given under "Zero-shot classification)--12 lists in total.

In [20]:
# These are the possible categories of relevance we have defined.
# Diversity and inclusion = 1
# Culture and values = 2
# Work life balance = 3
# Senior management = 4
# Career opportunities = 5
# Compensation and benefits = 6
candidate_labels = ['diversity and inclusion', 'culture and values', 'work life balance', 'senior management', 'career opportunities', 'compensation and benefits']
pro1 = []
con1 = []
pro2 = []
con2 = []
pro3 = []
con3 = []
pro4 = []
con4 = []
pro5 = []
con5 = []
pro6 = []
con6 = []

pros = [pro1, pro2, pro3, pro4, pro5, pro6]
cons = [con1, con2, con3, con4, con5, con6]

# Let's be picky and assume that if the top value is lower than 0.4, the
# sentence is not relevant.

def sortReviewSentencesUsingZeroShot(sentenceList, labeledContainers):
  for sentence in sentenceList:
    cat = classifier(sentence, candidate_labels)
    if float(cat['scores'][0]) > 0.4:
      label = cat['labels'][0]
      if label == candidate_labels[0]:
        labeledContainers[0].append(sentence)
      elif label == candidate_labels[1]:
        labeledContainers[1].append(sentence)
      elif label == candidate_labels[2]:
        labeledContainers[2].append(sentence)
      elif label == candidate_labels[3]:
        labeledContainers[3].append(sentence)
      elif label == candidate_labels[4]:
        labeledContainers[4].append(sentence)
      elif label == candidate_labels[5]:
        labeledContainers[5].append(sentence)

In [21]:
sortReviewSentencesUsingZeroShot(proSentences, pros)
print(pros)

KeyboardInterrupt: ignored

In [22]:
sortReviewSentencesUsingZeroShot(conSentences, cons)
print(cons)

[[' They give lip service to diversity, but without cleaning house of the misogynists in leadership, nothing will ever change', ' some were too idiosyncratic to get along with the others but mostly everyone likes everyone else', 'A lack of vision of innovation, benefiting from old well established games, a bit of an old boys club, D&I issues does the box ticking exercise rather than impacting real change', ' The diversity and employee engagement initiatives are half-baked and the HQ energy is horribly depressing for a gaming company', "It's not a highly diverse company, and the time for that is over", ' Many events will be "planned" and then cancelled which isn\'t necessarily anyone\'s fault but it proves that diversity and accepting those who are different really isn\'t a top priority', 'some equity issue during company'], ['Lack of any company culture', 'High expectations and goals for all teams', 'Time management Functional watering dishes Gospel', ' Not a place for latest and great

Now we'll print some classifiers and store them for our confusion matrix.

In [25]:
def printClassifiersForConfusion(sentenceList, howMany):
  count = howMany
  while count >= 0:
    print(classifier(sentenceList[count], candidate_labels))
    count -= 1

In [26]:
printClassifiersForConfusion(proSentences, 25)

{'sequence': ' free snacks', 'labels': ['compensation and benefits', 'work life balance', 'diversity and inclusion', 'culture and values', 'senior management', 'career opportunities'], 'scores': [0.3532989025115967, 0.22076530754566193, 0.18542596697807312, 0.13984614610671997, 0.05666486173868179, 0.0439988374710083]}
{'sequence': ' virtual position ', 'labels': ['career opportunities', 'work life balance', 'compensation and benefits', 'diversity and inclusion', 'culture and values', 'senior management'], 'scores': [0.27631133794784546, 0.25165843963623047, 0.1517743021249771, 0.14826814830303192, 0.11003493517637253, 0.06195275858044624]}
{'sequence': 'Fast pace flexible profesional culture', 'labels': ['career opportunities', 'culture and values', 'work life balance', 'diversity and inclusion', 'compensation and benefits', 'senior management'], 'scores': [0.546012818813324, 0.18102511763572693, 0.15644672513008118, 0.049793124198913574, 0.03349518030881882, 0.03322705253958702]}
{'s

In [27]:
printClassifiersForConfusion(conSentences, 25)

{'sequence': ' Testing can get repetitive as you are often checking no more than a few specific areas of a game', 'labels': ['work life balance', 'culture and values', 'compensation and benefits', 'diversity and inclusion', 'senior management', 'career opportunities'], 'scores': [0.2553131580352783, 0.23102599382400513, 0.18406148254871368, 0.14575156569480896, 0.09810975193977356, 0.08573805540800095]}
{'sequence': " Could be a pro if you need the money or don't have anything else going on in your life", 'labels': ['career opportunities', 'work life balance', 'compensation and benefits', 'diversity and inclusion', 'culture and values', 'senior management'], 'scores': [0.4046466648578644, 0.2524886727333069, 0.20046062767505646, 0.06140940263867378, 0.05354706197977066, 0.027447517961263657]}
{'sequence': ' Mandatory OT', 'labels': ['work life balance', 'culture and values', 'compensation and benefits', 'career opportunities', 'diversity and inclusion', 'senior management'], 'scores': 